# Kappa score calculation

In [2]:
# Project imports
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score

# Project variables
inputFile = "input.csv"

##### Import CSV
Import the CSV and drop collums that are not relevant for this script.

In [4]:
data = pd.read_csv(inputFile)

# Drop the index, thread and email ID
data.drop("#", inplace=True, axis=1)
data.drop("Tread Id", inplace=True, axis=1)
data.drop("Email ID", inplace=True, axis=1)
data.drop("Agreement?", inplace=True, axis=1)
data.drop("Decided", inplace=True, axis=1)

# Drop NaN value rows
data.dropna(inplace=True)

# Display
data.head(5)

,Marten,Rick,Krishan
0,not-ak,not-ak,not-ak
1,technology,not-ak,"technology,existence,existence-behavioral"
2,technology,not-ak,"technology,existence,existence-behavioral"
3,not-ak,not-ak,not-ak
4,process,process,process


In [18]:
# Get unique labels used in the input set.
# Used to verify validate validity of the data.
uniques = pd.unique(data.values.ravel("K"))
uniques = pd.Series(uniques).map(lambda x: x.split(',')).explode().unique()

# Display unqiues
for label in uniques:
    print(label)

not-ak
technology
process
existence
existence-structural
existence-behavioral
property
existence-behaviour


##### Convert to AK and Not-AK
Used for comparisons of Kappa score for checking if the group is on the same line for this property.

In [36]:
# Hierachy for most important to least important labels
labelHierachy = ["Executive", "Property", "Existence", "Non-architectural"]

# Mapping from labels in the tool to the labels in the hierachy.
# Labels that are not mapped may be ignored.
labelMapping = {
    "existence": labelHierachy.index("Existence"),
    "property": labelHierachy.index("Property"),
    "process": labelHierachy.index("Executive"),
    "technology": labelHierachy.index("Executive"),
    "not-ak": labelHierachy.index("Non-architectural")
}

def mapCel(cellData: str) -> str:
    importance = pd.Series(cellData.split(",")).map(labelMapping).dropna().astype(int).max()

    if importance == None:
        print("No importance found")
        return labelMapping["not-ak"]

    return labelHierachy[importance]

transformedData = data.applymap(mapCel)




,Marten,Rick,Krishan
0,Non-architectural,Non-architectural,Non-architectural
1,Executive,Non-architectural,Existence
2,Executive,Non-architectural,Existence
3,Non-architectural,Non-architectural,Non-architectural
4,Executive,Executive,Executive
...,...,...,...
99,Existence,Existence,Existence
100,Non-architectural,Non-architectural,Non-architectural
101,Existence,Existence,Existence
102,Non-architectural,Non-architectural,Non-architectural
